In [1]:
# write all code in one cell 

#========================Load data=========================
import numpy as np
import pandas as pd

train_data_source = 'C:\\Users\\Admin\\Dropbox\\Spring 2019\\Masters project\\Project\\Data\\agnews_train.csv'
test_data_source = 'C:\\Users\\Admin\\Dropbox\\Spring 2019\\Masters project\\Project\\Data\\agnews_valid.csv'

train_df = pd.read_csv(train_data_source)
test_df = pd.read_csv(test_data_source)

In [2]:
# to change use .astype() 
test_df['text'] = test_df.text.astype(str)
train_df['text'] = train_df.text.astype(str)

In [3]:
# train_df.head()
# test_df.dtypes

In [4]:
# convert string to lower case 
train_texts = train_df['text'].values 
train_texts = [s.lower() for s in train_texts] 

test_texts = test_df['text'].values 
test_texts = [s.lower() for s in test_texts] 

In [5]:
#=======================Convert string to index================
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)

C:\Users\Admin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
# Convert string to index 
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)

# Padding
train_data = pad_sequences(train_sequences, maxlen=1014, padding='post')
test_data = pad_sequences(test_texts, maxlen=1014, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')

#=======================Get classes================
train_classes = train_df['label'].values
train_class_list = [x-1 for x in train_classes]

test_classes = test_df['label'].values
test_class_list = [x-1 for x in test_classes]

from keras.utils import to_categorical
train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)

In [7]:
print(tk.word_index)

{'UNK': 1, ' ': 2, 'e': 3, 'a': 4, 't': 5, 'i': 6, 'o': 7, 's': 8, 'n': 9, 'r': 10, 'l': 11, 'd': 12, 'h': 13, 'c': 14, 'u': 15, 'p': 16, 'm': 17, 'g': 18, 'f': 19, 'y': 20, 'w': 21, 'b': 22, '.': 23, 'v': 24, 'k': 25, ',': 26, '-': 27, ';': 28, '3': 29, '0': 30, 'x': 31, '9': 32, 'j': 33, 'q': 34, '#': 35, '1': 36, '(': 37, ')': 38, '2': 39, "'": 40, 'z': 41, '&': 42, '\\': 43, ':': 44, '/': 45, '5': 46, '4': 47, '6': 48, '"': 49, '7': 50, '$': 51, '8': 52, '=': 53, '?': 54, '!': 55, '*': 56, '_': 57}


In [8]:
vocab_size=len(tk.word_index)
vocab_size

57

In [9]:
embedding_weights=[]

In [10]:
embedding_weights.append(np.zeros(vocab_size))

In [11]:
for char, i in tk.word_index.items(): # from index 1 to 70
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)

In [12]:
print(embedding_weights.shape) # first row all 0 for PAD, 69 char, last row for UNK
embedding_weights

(58, 57)


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [13]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [14]:
# parameter 
input_size = 1014
# vocab_size = 69
embedding_size = 57
conv_layers = [[256, 7, 3], 
               [256, 7, 3], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 4
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

In [15]:
# Embedding layer Initialization
embedding_layer = Embedding(vocab_size+1, 
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

In [16]:
# Model 

# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding 
x = embedding_layer(inputs)
# Conv 
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x) 
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x) # Final shape=(None, 34, 256)
x = Flatten()(x) # (None, 8704)
# Fully connected layers 
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x) # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy']) # Adam, categorical_crossentropy
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1014)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1014, 57)          3306      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1008, 256)         102400    
_________________________________________________________________
activation_1 (Activation)    (None, 1008, 256)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 336, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 330, 256)          459008    
_________________________________________________________________
activation_2 (Activation)    (None, 330, 256)          0         
__________

In [17]:
# 1000 training samples and 100 testing samples
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices][:]
y_train = train_classes[indices][:]

x_test = test_data[:]
y_test = test_classes[:]

In [18]:
# Training
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=6,
          verbose=2)

Train on 16800 samples, validate on 5160 samples
Epoch 1/6
 - 971s - loss: 1.3757 - acc: 0.2995 - val_loss: 1.3574 - val_acc: 0.3231
Epoch 2/6
 - 954s - loss: 1.2964 - acc: 0.3810 - val_loss: 1.1224 - val_acc: 0.4814
Epoch 3/6
 - 961s - loss: 0.9166 - acc: 0.5918 - val_loss: 0.7036 - val_acc: 0.6981
Epoch 4/6
 - 959s - loss: 0.6335 - acc: 0.7227 - val_loss: 0.5928 - val_acc: 0.7624
Epoch 5/6
 - 957s - loss: 0.4677 - acc: 0.8163 - val_loss: 0.6846 - val_acc: 0.7403
Epoch 6/6
 - 959s - loss: 0.3799 - acc: 0.8618 - val_loss: 0.4998 - val_acc: 0.8248


In [19]:
model.save("C:\\Users\\Admin\\Dropbox\\Spring 2019\\Masters project\\Project\\my_model_agnews.h5")